In [1]:
#!pip install tensorflow
#!pip install keras

In [2]:
# keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (batch_size=32,timesteps=1,features=1)))
# keras.layers.GRU(units = 50, return_sequences = True, input_shape = (batch_size=32,timesteps=1,features=1)))
# only 2 arguments in input shape indicate any batch size 

In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("MSFT.csv", na_values=['null'], index_col='Date', parse_dates=True, infer_datetime_format=True)
df.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061751,1031788800
1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063956,308160000
1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065059,133171200
1986-03-18,0.102431,0.103299,0.098958,0.099826,0.063405,67766400
1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062302,47894400


In [5]:
df.tail(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-04-27,261.579987,263.190002,260.119995,261.970001,261.970001,31014200
2021-04-28,256.079987,256.540009,252.949997,254.559998,254.559998,46903100
2021-04-29,255.460007,256.100006,249.000000,252.509995,252.509995,40589000
2021-04-30,249.740005,253.080002,249.600006,252.179993,252.179993,30929200
2021-05-03,253.399994,254.350006,251.119995,251.860001,251.860001,19598900


In [6]:
df.shape

(8857, 6)

In [7]:
#Set Target Variable
high = df['High']

In [8]:
# the following example is for 1 feature, 60 timesteps
X = []
y = []
for i in range(60, df.shape[0]):
    X.append(high.iloc[i-60:i])    # 0-59 rows
    y.append(high[i])              # 60th row
X = np.array(X)
y = np.array(y)
print(X[0], y[0])

[0.101563 0.102431 0.103299 0.103299 0.100694 0.09809  0.097222 0.092882
 0.092014 0.095486 0.096354 0.096354 0.095486 0.097222 0.098958 0.097222
 0.097222 0.097222 0.09809  0.098958 0.101563 0.101563 0.100694 0.105035
 0.105035 0.105035 0.102431 0.101563 0.100694 0.111979 0.121962 0.118924
 0.118056 0.115451 0.111979 0.111979 0.110243 0.111979 0.111111 0.111111
 0.111111 0.113715 0.112847 0.111979 0.112847 0.114583 0.111979 0.110243
 0.110243 0.108507 0.109375 0.111111 0.114583 0.118924 0.123264 0.121528
 0.118056 0.118924 0.118924 0.118924] 0.1189239993691444


In [9]:
X.shape

(8797, 60)

In [10]:
y.shape

(8797,)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [12]:
X_train.shape

(7037, 60)

In [13]:
X_test.shape

(1760, 60)

In [14]:
# add one more dimension to X_train i.e. (9023, 60) to (9023,60,1)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [15]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
# LSTM input_shape(60,1) means number_of_timesteps=60, features=1)
# input_shape(batch_size,timesteps,features), only 2 arguments indicate any batch size 
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units = 1))     # as we are dealing with one feature at a time


model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint saves the model while monitoring a specific parameter of the model. 
# In this case we are monitoring loss. 
# The model will only be saved if the loss in current epoch is less than in the last epoch.
checkpoint = ModelCheckpoint("lstm_stock.h5", monitor='loss', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')

# EarlyStopping stops training of the model early if there is no increase in the monitored parameter. 
# In this case we are monitoring loss. 
# patience=20 means that model will stop to train if it doesn’t see any decrease in loss in 20 epochs.
early = EarlyStopping(monitor='loss', min_delta=0, patience=20, verbose=1, mode='auto')

model.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[checkpoint,early])      

Epoch 1/100
220/220 [==============================] - ETA: 0s - loss: 2444.2478
Epoch 1: loss improved from inf to 2444.24780, saving model to lstm_stock.h5
220/220 [==============================] - 17s 54ms/step - loss: 2444.2478
Epoch 2/100
220/220 [==============================] - ETA: 0s - loss: 1973.8534
Epoch 2: loss improved from 2444.24780 to 1973.85339, saving model to lstm_stock.h5
220/220 [==============================] - 12s 52ms/step - loss: 1973.8534
Epoch 3/100
220/220 [==============================] - ETA: 0s - loss: 1697.7389
Epoch 3: loss improved from 1973.85339 to 1697.73889, saving model to lstm_stock.h5
220/220 [==============================] - 11s 52ms/step - loss: 1697.7389
Epoch 4/100
220/220 [==============================] - ETA: 0s - loss: 1486.4294
Epoch 4: loss improved from 1697.73889 to 1486.42944, saving model to lstm_stock.h5
220/220 [==============================] - 11s 51ms/step - loss: 1486.4294
Epoch 5/100
219/220 [==========================

220/220 [==============================] - 12s 54ms/step - loss: 44.8576
Epoch 36/100
220/220 [==============================] - ETA: 0s - loss: 44.8988
Epoch 36: loss did not improve from 44.85764
220/220 [==============================] - 12s 54ms/step - loss: 44.8988
Epoch 37/100
220/220 [==============================] - ETA: 0s - loss: 39.6423
Epoch 37: loss improved from 44.85764 to 39.64232, saving model to lstm_stock.h5
220/220 [==============================] - 12s 54ms/step - loss: 39.6423
Epoch 38/100
220/220 [==============================] - ETA: 0s - loss: 38.4015
Epoch 38: loss improved from 39.64232 to 38.40149, saving model to lstm_stock.h5
220/220 [==============================] - 12s 54ms/step - loss: 38.4015
Epoch 39/100
220/220 [==============================] - ETA: 0s - loss: 37.4402
Epoch 39: loss improved from 38.40149 to 37.44020, saving model to lstm_stock.h5
220/220 [==============================] - 12s 54ms/step - loss: 37.4402
Epoch 40/100
220/220 [=====

220/220 [==============================] - 12s 55ms/step - loss: 26.8062
Epoch 75/100
219/220 [============================>.] - ETA: 0s - loss: 22.1447
Epoch 75: loss improved from 22.31797 to 22.30469, saving model to lstm_stock.h5
220/220 [==============================] - 12s 54ms/step - loss: 22.3047
Epoch 76/100
220/220 [==============================] - ETA: 0s - loss: 22.2480
Epoch 76: loss improved from 22.30469 to 22.24800, saving model to lstm_stock.h5
220/220 [==============================] - 12s 55ms/step - loss: 22.2480
Epoch 77/100
220/220 [==============================] - ETA: 0s - loss: 26.1208
Epoch 77: loss did not improve from 22.24800
220/220 [==============================] - 12s 54ms/step - loss: 26.1208
Epoch 78/100
220/220 [==============================] - ETA: 0s - loss: 22.3527
Epoch 78: loss did not improve from 22.24800
220/220 [==============================] - 12s 54ms/step - loss: 22.3527
Epoch 79/100
220/220 [==============================] - ETA: 0s

In [17]:
y_pred = model.predict(X_test)
y_pred

55/55 [==============================] - 1s 13ms/step


array([[ 0.0618515],
       [ 1.9811678],
       [27.031998 ],
       ...,
       [17.78685  ],
       [ 8.598194 ],
       [23.149738 ]], dtype=float32)